<a href="https://colab.research.google.com/github/omier/NLP-final-project/blob/master/NLP_Final_Project.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Init Notebook

In [1]:
!git clone https://github.com/omier/NLP-final-project.git

Cloning into 'NLP-final-project'...
remote: Enumerating objects: 18, done.
remote: Counting objects: 100% (18/18), done.
remote: Compressing objects: 100% (16/16), done.
remote: Total 18 (delta 5), reused 4 (delta 1), pack-reused 0
Unpacking objects: 100% (18/18), done.


In [2]:
!pip install -qq transformers

     |████████████████████████████████| 1.8MB 16.8MB/s 
     |████████████████████████████████| 890kB 37.3MB/s 
     |████████████████████████████████| 3.2MB 60.8MB/s 


In [3]:
import transformers
from transformers import BertModel, BertTokenizer, AdamW, get_linear_schedule_with_warmup
import torch
import numpy as np
import pandas as pd
from torch import nn, optim
from torch.utils.data import Dataset, DataLoader
from sklearn.model_selection import train_test_split
import plotly.express as px
import pprint

import warnings
warnings.filterwarnings('ignore')

In [4]:
RANDOM_SEED = 42
np.random.seed(RANDOM_SEED)
torch.manual_seed(RANDOM_SEED)

In [5]:
device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
device

device(type='cuda', index=0)

# Import BERT

In [6]:
PRE_TRAINED_MODEL_NAME = 'distilbert-base-uncased'

In [7]:
tokenizer = BertTokenizer.from_pretrained(PRE_TRAINED_MODEL_NAME)

In [8]:
bert_model = BertModel.from_pretrained(PRE_TRAINED_MODEL_NAME).to(device)

Some weights of the model checkpoint at distilbert-base-uncased were not used when initializing BertModel: ['distilbert.embeddings.word_embeddings.weight', 'distilbert.embeddings.position_embeddings.weight', 'distilbert.embeddings.LayerNorm.weight', 'distilbert.embeddings.LayerNorm.bias', 'distilbert.transformer.layer.0.attention.q_lin.weight', 'distilbert.transformer.layer.0.attention.q_lin.bias', 'distilbert.transformer.layer.0.attention.k_lin.weight', 'distilbert.transformer.layer.0.attention.k_lin.bias', 'distilbert.transformer.layer.0.attention.v_lin.weight', 'distilbert.transformer.layer.0.attention.v_lin.bias', 'distilbert.transformer.layer.0.attention.out_lin.weight', 'distilbert.transformer.layer.0.attention.out_lin.bias', 'distilbert.transformer.layer.0.sa_layer_norm.weight', 'distilbert.transformer.layer.0.sa_layer_norm.bias', 'distilbert.transformer.layer.0.ffn.lin1.weight', 'distilbert.transformer.layer.0.ffn.lin1.bias', 'distilbert.transformer.layer.0.ffn.lin2.weight', 'd

In [9]:
bert_model

BertModel(
  (embeddings): BertEmbeddings(
    (word_embeddings): Embedding(30522, 768, padding_idx=0)
    (position_embeddings): Embedding(512, 768)
    (token_type_embeddings): Embedding(2, 768)
    (LayerNorm): LayerNorm((768,), eps=1e-12, elementwise_affine=True)
    (dropout): Dropout(p=0.1, inplace=False)
  )
  (encoder): BertEncoder(
    (layer): ModuleList(
      (0): BertLayer(
        (attention): BertAttention(
          (self): BertSelfAttention(
            (query): Linear(in_features=768, out_features=768, bias=True)
            (key): Linear(in_features=768, out_features=768, bias=True)
            (value): Linear(in_features=768, out_features=768, bias=True)
            (dropout): Dropout(p=0.1, inplace=False)
          )
          (output): BertSelfOutput(
            (dense): Linear(in_features=768, out_features=768, bias=True)
            (LayerNorm): LayerNorm((768,), eps=1e-12, elementwise_affine=True)
            (dropout): Dropout(p=0.1, inplace=False)
          

# Load Data + features extraction using BERT

In [68]:
import nltk
import string
import numpy as np
import pandas as pd
from nltk.corpus import stopwords
import re

In [66]:
nltk.download('stopwords')

print(stopwords.words('english')[:15])

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
['i', 'me', 'my', 'myself', 'we', 'our', 'ours', 'ourselves', 'you', "you're", "you've", "you'll", "you'd", 'your', 'yours']


In [100]:
df = pd.read_csv("NLP-final-project/Data/Combined_News_DJIA.csv")

In [101]:
df = df.drop(['Date'], axis=1)
tops = df.columns[1:]

df['headline'] = df[tops[:1]].apply(lambda row:'.'.join(row.values.astype(str)), axis=1)
df = df.drop(tops, axis=1)

df['label'] = df['Label']
df = df.drop(['Label'], axis=1)

# remove special characters
df = df.replace('b\"|b\'|\\\\|\\\"', '', regex=True)

df.head()

,headline,label
0,Georgia 'downs two Russian warplanes' as count...,0
1,Why wont America and Nato help us? If they won...,1
2,Remember that adorable 9-year-old who sang at ...,0
3,U.S. refuses Israel weapons to attack Iran: r...,0
4,All the experts admit that we should legalise ...,1


In [102]:
def clean_text(text):
    
    ## Remove puncuation
    text = text.translate(string.punctuation)
    
    ## Convert words to lower case and split them
    text = text.lower().split()
    
    ## Remove stop words
    stops = set(stopwords.words("english"))
    text = [w for w in text if not w in stops]
    
    text = " ".join(text)
    ## Clean the text
    text = re.sub(r"[^A-Za-z0-9^,!.\/'+-=]", " ", text)
    text = re.sub(r"what's", "what is ", text)
    text = re.sub(r"\'s", " ", text)
    text = re.sub(r"\'ve", " have ", text)
    text = re.sub(r"n't", " not ", text)
    text = re.sub(r"i'm", "i am ", text)
    text = re.sub(r"\'re", " are ", text)
    text = re.sub(r"\'d", " would ", text)
    text = re.sub(r"\'ll", " will ", text)
    text = re.sub(r",", " ", text)
    text = re.sub(r"\.", " ", text)
    text = re.sub(r"!", " ! ", text)
    text = re.sub(r"\/", " ", text)
    text = re.sub(r"\^", " ^ ", text)
    text = re.sub(r"\+", " + ", text)
    text = re.sub(r"\-", " - ", text)
    text = re.sub(r"\=", " = ", text)
    text = re.sub(r"'", " ", text)
    text = re.sub(r"(\d+)(k)", r"\g<1>000", text)
    text = re.sub(r":", " : ", text)
    text = re.sub(r" e g ", " eg ", text)
    text = re.sub(r" b g ", " bg ", text)
    text = re.sub(r"u s", "american", text)
    text = re.sub(r"\0s", "0", text)
    text = re.sub(r" 9 11 ", "911", text)
    text = re.sub(r"e - mail", "email", text)
    text = re.sub(r"j k", "jk", text)
    text = re.sub(r"\s{2,}", " ", text)
    ## Stemming
    text = text.split()
    stemmer = nltk.stem.SnowballStemmer('english')
    stemmed_words = [stemmer.stem(word) for word in text]
    text = " ".join(stemmed_words)

    return text

In [14]:
def punctuation_stopwords_removal(headlines):
    # filters charecter-by-charecter
    remove_punctuation = [ch for ch in headlines if ch not in string.punctuation]
    # convert them back to sentences and split into words
    remove_punctuation = "".join(remove_punctuation).split()
    return remove_punctuation

    # # remove stop words
    # filtered_headlines = [word for word in remove_punctuation if word.lower() not in stopwords.words('english')]
    # return filtered_headlines

In [103]:
df.loc[:, 'headline'] = df['headline'].apply(clean_text)

In [104]:
df.head()

,headline,label
0,georgia would own two russian warplan countri ...,0
1,wont america nato help us wont help us now hel...,1
2,rememb ador 9 - year - old sang open ceremoni ...,0
3,american refus israel weapon attack iran : report,0
4,expert admit legalis drug,1


In [138]:
# input_ids = tokenizer.encode(list(df['headline']), is_split_into_words=True, padding=True)

encoding = tokenizer.encode(
  list(df['headline']),
  is_split_into_words=True,
  padding=True,
  return_tensors='pt'
)

# tokenized = df['headline'].apply((lambda x: tokenizer.encode(x)))
# max_len = 0
# for i in tokenized.values:
#     if len(i)>max_len:
#         max_len = len(i)
# print(max_len)

In [141]:
len(df)

1989

In [140]:
encoding.shape

torch.Size([1, 38040])

In [18]:
padded = np.array([i + [0]*(max_len - len(i)) for i in tokenized.values])
attention_mask = np.where(padded != 0, 1, 0)

input_ids = torch.tensor(padded).to(device)
attention_mask = torch.tensor(attention_mask).to(device)
labels = torch.tensor(df['label'].to_numpy(), dtype=torch.float).to(device)

In [19]:
with torch.no_grad():
    features = bert_model(input_ids, attention_mask=attention_mask).pooler_output

In [20]:
train_features, test_features, train_labels, test_labels = train_test_split(features, labels, test_size=0.2, random_state=RANDOM_SEED)
test_features, val_features, test_labels, val_labels = train_test_split(test_features, test_labels, test_size=0.5, random_state=RANDOM_SEED)

In [21]:
class HeadlinesStocksDataset(Dataset):
  def __init__(self, features, labels):
    self.features = features
    self.labels = labels

  def __len__(self):
    return len(self.features)

  def __getitem__(self, item):
    return self.features[item], self.labels[item]

In [22]:
BATCH_SIZE = 8

def create_data_loader(features, labels, batch_size, shuffle=False):
  return DataLoader(
    HeadlinesStocksDataset(features, labels),
    batch_size=batch_size,
    shuffle=shuffle
  )

In [23]:
train_data_loader = create_data_loader(train_features, train_labels, BATCH_SIZE, shuffle=True)
val_data_loader = create_data_loader(val_features, val_labels, BATCH_SIZE)
test_data_loader = create_data_loader(test_features, test_labels, BATCH_SIZE)

In [24]:
data_loaders = {'train': train_data_loader, 'validation': val_data_loader, 'test': test_data_loader}

In [25]:
for set_name, loader in data_loaders.items():
  print(f'{set_name}: {len(loader.dataset)}')

train: 1591
validation: 199
test: 199


# Helpers

In [26]:
def train_epoch(model, data_loader, loss_fn, optimizer, scheduler):
  losses = []
  correct_predictions = 0
  
  for features, labels in data_loader:
    features = features.to(device)
    labels = labels.to(device)

    outputs = torch.squeeze(model(features))

    # for accuracy calculation
    predicted_labels = (outputs > 0.5).float() * 1
    correct_predictions += torch.sum(predicted_labels == labels)

    # calculate and save loss
    loss = loss_fn(outputs, labels)
    losses.append(loss.item())

    # optimizer step
    loss.backward()
    nn.utils.clip_grad_norm_(model.parameters(), max_norm=1.0)
    optimizer.step()
    if scheduler:
      scheduler.step()
    optimizer.zero_grad()
  
  return {'accuracy': correct_predictions.item() / len(data_loader.dataset),
          'loss': np.mean(losses)}

In [27]:
def eval_model(model, loss_fn, sets=['validation']):
  with torch.no_grad():
    sets_metrics = dict()

    for set_name, dataloader in data_loaders.items():
      if set_name in sets:
        losses = []
        correct_predictions = 0
  
        for features, labels in dataloader:
          features = features.to(device)
          labels = labels.to(device)

          outputs = torch.squeeze(model(features))

          # for accuracy calculation
          predicted_labels = (outputs > 0.5).float() * 1
          correct_predictions += torch.sum(predicted_labels == labels)

          # calculate and save loss
          losses.append(loss_fn(outputs, labels).item())

        sets_metrics[set_name] = {'accuracy': correct_predictions.item() / len(dataloader.dataset),
                                  'loss': np.mean(losses)}

    return sets_metrics


In [28]:
def train(model, n_epochs, train_data_loader, loss_fn, optimizer, scheduler):
  history = []

  for epoch in range(n_epochs):
    print(f'Epoch {epoch + 1}/{n_epochs} Metrics')
    
    current_metrics = { 'train': 
      train_epoch(
        model,
        train_data_loader,
        loss_fn,
        optimizer,
        scheduler
      ) 
    }
    current_metrics.update(eval_model(model, loss_fn))

    pprint.pprint(current_metrics, indent=4)
    print('-' * 10)

    history.append(current_metrics)

  return history

In [29]:
def plot(history):
  metrics_map = dict()

  for e_sets in history:
    for set_name, set_metrics in e_sets.items():
      for metric_name, metric_value in set_metrics.items():

        if metric_name not in metrics_map:
          metrics_map[metric_name] = dict()
        if set_name not in metrics_map[metric_name]:
          metrics_map[metric_name][set_name] = []

        metrics_map[metric_name][set_name].append(metric_value)
      
  for metric_name, sets in metrics_map.items():
    df = None
    for set_name, set_metrics in sets.items():
      size = len(set_metrics)
      if df is None:
        df = pd.DataFrame({"epoch": np.linspace(1, size, size),
                      metric_name: set_metrics,
                      "set": [set_name] * size})
      else:
        df = df.append(pd.DataFrame({"epoch": np.linspace(1, size, size),
                      metric_name: set_metrics,
                      "set": [set_name] * size}), ignore_index=True)

    fig = px.line(df, x="epoch", y=metric_name, line_group="set", title=f"epoch {metric_name} per dataset", color="set", hover_name="set")
    fig.show()

# Logistic Regression Classifier

In [30]:
class LogisticClassifier(nn.Module):
  def __init__(self):
    super(LogisticClassifier, self).__init__()
    self.classifier = nn.Sequential(nn.Linear(bert_model.config.hidden_size, 1),
                                    nn.Sigmoid())

  def forward(self, x):
    return self.classifier(x)  


In [31]:
model = LogisticClassifier().to(device)

In [32]:
EPOCHS = 10
optimizer = AdamW(model.parameters(), lr=2e-5, correct_bias=False)
total_steps = len(train_data_loader) * EPOCHS
scheduler = get_linear_schedule_with_warmup(
  optimizer,
  num_warmup_steps=0,
  num_training_steps=total_steps
)
loss_fn = nn.BCELoss().to(device)

In [33]:
%%time
history = train(model, EPOCHS, train_data_loader, loss_fn, optimizer, scheduler)

Epoch 1/10 Metrics
{   'train': {'accuracy': 0.5147705845380264, 'loss': 0.6942029247930901},
    'validation': {'accuracy': 0.5728643216080402, 'loss': 0.688151569366455}}
----------
Epoch 2/10 Metrics
{   'train': {'accuracy': 0.5279698302954117, 'loss': 0.6928458705020311},
    'validation': {'accuracy': 0.5728643216080402, 'loss': 0.6904939556121826}}
----------
Epoch 3/10 Metrics
{   'train': {'accuracy': 0.524198617221873, 'loss': 0.692365534341515},
    'validation': {'accuracy': 0.5879396984924623, 'loss': 0.6845856642723084}}
----------
Epoch 4/10 Metrics
{   'train': {'accuracy': 0.5267127592708988, 'loss': 0.69214449575798},
    'validation': {'accuracy': 0.5879396984924623, 'loss': 0.6837685680389405}}
----------
Epoch 5/10 Metrics
{   'train': {'accuracy': 0.5260842237586424, 'loss': 0.6919403741108113},
    'validation': {'accuracy': 0.5879396984924623, 'loss': 0.684586546421051}}
----------
Epoch 6/10 Metrics
{   'train': {'accuracy': 0.5273412947831553, 'loss': 0.691774

In [34]:
plot(history)

In [ ]:
eval_model(model, loss_fn, sets=['test'])

{'test': {'accuracy': 0.5527638190954773, 'loss': 0.6909364628791809}}

# Naive Bayes

In [ ]:
from sklearn.naive_bayes import GaussianNB

gnb = GaussianNB()

In [ ]:
X_train = train_features.cpu().detach().numpy()
X_test = test_features.cpu().detach().numpy()
y_train = train_labels.cpu().detach().numpy()
y_test = test_labels.cpu().detach().numpy()

In [ ]:
y_pred = gnb.fit(X_train, y_train).predict(X_test)
print("Number of mislabeled points out of a total %d points : %d" % (X_test.shape[0], (y_test != y_pred).sum()))

Number of mislabeled points out of a total 199 points : 96


# LSTM

In [142]:
from torchtext import data    

In [289]:
df = pd.read_csv("NLP-final-project/Data/Combined_News_DJIA.csv")

In [290]:
df = df.drop(['Date'], axis=1)
tops = df.columns[1:]

df['headline'] = df[tops[:1]].apply(lambda row:'.'.join(row.values.astype(str)), axis=1)
df = df.drop(tops, axis=1)

df['label'] = df['Label']
df = df.drop(['Label'], axis=1)

# remove special characters
df = df.replace('b\"|b\'|\\\\|\\\"', '', regex=True)

df.head()

,headline,label
0,Georgia 'downs two Russian warplanes' as count...,0
1,Why wont America and Nato help us? If they won...,1
2,Remember that adorable 9-year-old who sang at ...,0
3,U.S. refuses Israel weapons to attack Iran: r...,0
4,All the experts admit that we should legalise ...,1


In [292]:
df.to_csv("headlines.csv")

In [365]:
TEXT = data.Field(tokenize='spacy',batch_first=True,include_lengths=True)
LABEL = data.LabelField(dtype = torch.float,batch_first=True)

In [366]:
fields = [(None, None), ('headline', TEXT), ('label', LABEL)]

In [367]:
training_data = data.TabularDataset(path="headlines.csv", format='csv', fields=fields, skip_header=True)

In [369]:
vars(training_data.examples[0])

{'headline': ['Georgia',
  "'",
  'downs',
  'two',
  'Russian',
  'warplanes',
  "'",
  'as',
  'countries',
  'move',
  'to',
  'brink',
  'of',
  'war'],
 'label': '0'}

In [370]:
train_data, valid_data = training_data.split(split_ratio=0.7)

In [371]:
TEXT.build_vocab(train_data,min_freq=3,vectors = "glove.6B.100d")  
LABEL.build_vocab(train_data)

In [372]:
#No. of unique tokens in text
print("Size of TEXT vocabulary:",len(TEXT.vocab))

#No. of unique tokens in label
print("Size of LABEL vocabulary:",len(LABEL.vocab))

#Commonly used words
print(TEXT.vocab.freqs.most_common(10))  

#Word dictionary
print(TEXT.vocab.stoi)
print(LABEL.vocab.itos)

Size of TEXT vocabulary: 1871
Size of LABEL vocabulary: 2
[('the', 910), ('to', 858), ('.', 797), (',', 786), ('of', 706), ('in', 616), ('a', 507), ("'", 482), ('and', 440), ('-', 416)]
defaultdict(<function _default_unk_index at 0x7f5da0b1ed08>, {'<unk>': 0, '<pad>': 1, 'the': 2, 'to': 3, '.': 4, ',': 5, 'of': 6, 'in': 7, 'a': 8, "'": 9, 'and': 10, '-': 11, 'for': 12, 'is': 13, ':': 14, "'s": 15, 'that': 16, 'on': 17, 'The': 18, 'has': 19, 'by': 20, 'from': 21, 'with': 22, 'as': 23, 'it': 24, 'be': 25, 'have': 26, 'are': 27, 'an': 28, 'was': 29, 'who': 30, 'their': 31, 'will': 32, 'A': 33, 'at': 34, 'after': 35, ' ': 36, 'been': 37, 'not': 38, 'US': 39, 'his': 40, 'they': 41, ')': 42, '$': 43, '(': 44, 'all': 45, 'over': 46, 'its': 47, 'government': 48, 'says': 49, 'world': 50, 'he': 51, 'people': 52, 'I': 53, ';': 54, '?': 55, 'old': 56, 'than': 57, 'up': 58, 'years': 59, 'about': 60, 'out': 61, 'police': 62, 'into': 63, '%': 64, 'them': 65, 'Israel': 66, 'first': 67, 'do': 68, 'new'

In [373]:
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

BATCH_SIZE = 16
#Load an iterator
train_iterator, valid_iterator = data.BucketIterator.splits(
    (train_data, valid_data), 
    batch_size = BATCH_SIZE,
    sort_key = lambda x: len(x.headline),
    sort_within_batch=True,
    device = device)

In [374]:
class classifier(nn.Module):
    
    #define all the layers used in model
    def __init__(self, vocab_size, embedding_dim, hidden_dim, output_dim, n_layers, 
                 bidirectional, dropout):
        
        #Constructor
        super().__init__()          
        
        #embedding layer
        self.embedding = nn.Embedding(vocab_size, embedding_dim)
        
        #lstm layer
        self.lstm = nn.LSTM(embedding_dim, 
                           hidden_dim, 
                           num_layers=n_layers, 
                           bidirectional=bidirectional, 
                           dropout=dropout,
                           batch_first=True)
        
        #dense layer
        self.fc = nn.Linear(hidden_dim * (2 if bidirectional else 1), output_dim)
        
        #activation function
        self.act = nn.Sigmoid()
        
    def forward(self, text, text_lengths):
        
        #text = [batch size,sent_length]
        embedded = self.embedding(text)
        #embedded = [batch size, sent_len, emb dim]
      
        #packed sequence
        packed_embedded = nn.utils.rnn.pack_padded_sequence(embedded, text_lengths,batch_first=True)
        
        packed_output, (hidden, cell) = self.lstm(packed_embedded)
        #hidden = [batch size, num layers * num directions,hid dim]
        #cell = [batch size, num layers * num directions,hid dim]
        
        #concat the final forward and backward hidden state
        hidden = torch.cat((hidden[-2,:,:], hidden[-1,:,:]), dim = 1)
                
        #hidden = [batch size, hid dim * num directions]
        dense_outputs=self.fc(hidden)

        #Final activation function
        outputs=self.act(dense_outputs)
        
        return outputs

In [375]:
#define hyperparameters
size_of_vocab = len(TEXT.vocab)
embedding_dim = 100
num_hidden_nodes = 256
num_output_nodes = 1
num_layers = 2
bidirection = True
dropout = 0.2

In [385]:
#instantiate the model
model = classifier(size_of_vocab, embedding_dim, num_hidden_nodes,num_output_nodes, num_layers, 
                   bidirectional = True, dropout = dropout).to(device)

In [386]:
optimizer = optim.Adam(model.parameters())
criterion = nn.BCELoss().to(device)

In [383]:
def train(model, iterator, optimizer, criterion):
    
    #initialize every epoch 
    epoch_loss = 0
    epoch_acc = 0
    
    #set the model in training phase
    model.train()  
    
    for batch in iterator:
        
        #resets the gradients after every batch
        optimizer.zero_grad()   
        
        #retrieve text and no. of words
        text, text_lengths = batch.headline
        text_lengths = text_lengths.to("cpu")
        
        #convert to 1D tensor
        predictions = model(text, text_lengths).squeeze()
        
        #compute the loss
        loss = criterion(predictions, batch.label)        
        
        #compute the binary accuracy
        acc = binary_accuracy(predictions, batch.label)   
        
        #backpropage the loss and compute the gradients
        loss.backward()       
        
        #update the weights
        optimizer.step()      
        
        #loss and accuracy
        epoch_loss += loss.item()  
        epoch_acc += acc.item()    
        
    return epoch_loss / len(iterator), epoch_acc / len(iterator)

In [379]:
def evaluate(model, iterator, criterion):
    
    #initialize every epoch
    epoch_loss = 0
    epoch_acc = 0

    #deactivating dropout layers
    model.eval()
    
    #deactivates autograd
    with torch.no_grad():
    
        for batch in iterator:
            #retrieve text and no. of words
            text, text_lengths = batch.headline
            text_lengths = text_lengths.to("cpu")
        
            #convert to 1d tensor
            predictions = model(text, text_lengths).squeeze()
                      
            #compute loss and accuracy
            loss = criterion(predictions, batch.label)
            acc = binary_accuracy(predictions, batch.label)
            
            #keep track of loss and accuracy
            epoch_loss += loss.item()
            epoch_acc += acc.item()
        
    return epoch_loss / len(iterator), epoch_acc / len(iterator)

In [380]:
def binary_accuracy(preds, y):
    pred_labels = (preds > 0.5).float() * 1
    correct = (pred_labels == y).float()
    acc = correct.sum() / len(correct)
    return acc

In [387]:
N_EPOCHS = 5
best_valid_loss = float('inf')

for epoch in range(N_EPOCHS):
     
    #train the model
    train_loss, train_acc = train(model, train_iterator, optimizer, criterion)
    
    #evaluate the model
    valid_loss, valid_acc = evaluate(model, valid_iterator, criterion)
    
    #save the best model
    if valid_loss < best_valid_loss:
        best_valid_loss = valid_loss
        torch.save(model.state_dict(), 'saved_weights.pt')
    
    print(f'\tTrain Loss: {train_loss:.3f} | Train Acc: {train_acc*100:.2f}%')
    print(f'\t Val. Loss: {valid_loss:.3f} |  Val. Acc: {valid_acc*100:.2f}%')


	Train Loss: 0.695 | Train Acc: 51.87%
	 Val. Loss: 0.697 |  Val. Acc: 46.48%
	Train Loss: 0.674 | Train Acc: 57.11%
	 Val. Loss: 0.715 |  Val. Acc: 47.76%
	Train Loss: 0.613 | Train Acc: 66.81%
	 Val. Loss: 0.772 |  Val. Acc: 48.13%
	Train Loss: 0.462 | Train Acc: 79.31%
	 Val. Loss: 0.989 |  Val. Acc: 48.06%
	Train Loss: 0.257 | Train Acc: 90.30%
	 Val. Loss: 1.251 |  Val. Acc: 47.43%


# OLD LSTM

In [77]:
df.head()

,headline,label
0,georgia would own two russian warplan countri ...,0
1,wont america nato help us wont help now help iraq,1
2,rememb ador 9 - year - old sang open ceremoni ...,0
3,u s refus israel weapon attack iran : report,0
4,expert admit legalis drug,1


In [86]:
corpus = []

max_len = 0 
for index, word_list in df["headline"].iteritems():
  max_len = max(max_len, len(word_list))
  corpus.extend(word_list)

vocab_to_int = {w:i+1 for i, w in enumerate(set(corpus))}

In [87]:
df.loc[:, 'headline'] = df['headline'].apply(lambda headline: [vocab_to_int[word] for word in headline] + [0]*(max_len - len(headline)))

In [88]:
df.head()

,headline,label
0,"[2, 1, 37, 40, 2, 38, 39, 14, 10, 37, 32, 12, ...",0
1,"[10, 37, 29, 34, 14, 39, 6, 1, 40, 38, 23, 39,...",1
2,"[40, 1, 6, 1, 6, 19, 14, 39, 9, 37, 40, 14, 26...",0
3,"[32, 14, 3, 14, 40, 1, 27, 32, 3, 14, 38, 3, 4...",0
4,"[1, 35, 16, 1, 40, 34, 14, 39, 9, 6, 38, 34, 1...",1


In [89]:
headlines = [item for sublist in df['headline'].tolist() for item in sublist]
features = torch.LongTensor(headlines).reshape(len(df), -1).to(device)
labels = torch.FloatTensor(df['label'].to_numpy()).to(device)

In [90]:
print(features.shape, labels.shape)

torch.Size([1989, 216]) torch.Size([1989])


In [91]:
train_features, test_features, train_labels, test_labels = train_test_split(features, labels, test_size=0.2, random_state=RANDOM_SEED)
test_features, val_features, test_labels, val_labels = train_test_split(test_features, test_labels, test_size=0.5, random_state=RANDOM_SEED)

In [92]:
train_data_loader = create_data_loader(train_features, train_labels, BATCH_SIZE, shuffle=True)
val_data_loader = create_data_loader(val_features, val_labels, BATCH_SIZE)
test_data_loader = create_data_loader(test_features, test_labels, BATCH_SIZE)

In [93]:
# Instantiate the model w/ hyperparams
vocab_size = len(vocab_to_int)+1 # +1 for the 0 padding
output_size = 1
embedding_dim = 400
hidden_dim = 256
n_layers = 2

In [94]:
class SentimentLSTM(nn.Module):
    """
    The RNN model that will be used to perform Sentiment analysis.
    """

    def __init__(self, vocab_size, output_size, embedding_dim, hidden_dim, n_layers, directions=1, drop_prob=0.5):
        """
        Initialize the model by setting up the layers.
        """
        super().__init__()

        self.output_size = output_size
        self.n_layers = n_layers
        self.hidden_dim = hidden_dim
        self.directions = directions
        self.bidirectional = (directions == 2)
        
        # embedding and LSTM layers
        self.embedding = nn.Embedding(vocab_size, embedding_dim)
        self.lstm = nn.LSTM(embedding_dim, hidden_dim, n_layers, 
                            dropout=drop_prob, batch_first=True, bidirectional=self.bidirectional)
        
        # dropout layer
        self.dropout = nn.Dropout(0.3)

        # linear and sigmoid layers
        self.fc = nn.Linear(hidden_dim * directions, output_size)
        self.sig = nn.Sigmoid()
        

    def forward(self, x):
        """
        Perform a forward pass of our model on some input and hidden state.
        """
        batch_size = x.size(0)

        self.init_hidden(batch_size)

        # embeddings and lstm_out
        embedded = self.embedding(x.long())
        lstm_out, self.hidden = self.lstm(embedded, self.hidden)
      
        # stack up lstm outputs
        lstm_out = lstm_out.contiguous().view(-1, self.hidden_dim * self.directions)
        
        # dropout and fully-connected layer
        out = self.dropout(lstm_out)
        out = self.fc(out)
        # sigmoid function
        sig_out = self.sig(out)
        
        # reshape to be batch_size first
        sig_out = sig_out.view(batch_size, -1)
        sig_out = sig_out[:, -1] # get last batch of labels
        
        # return last sigmoid output
        return sig_out

    def init_hidden(self, batch_size):
      self.hidden = (torch.zeros(self.n_layers * self.directions, batch_size, self.hidden_dim).to(device=device),
                     torch.zeros(self.n_layers * self.directions, batch_size, self.hidden_dim).to(device=device))

In [95]:
LSTM_model = SentimentLSTM(vocab_size, output_size, embedding_dim, hidden_dim, n_layers, directions=2, drop_prob=0.05).to(device)

In [96]:
LSTM_model

SentimentLSTM(
  (embedding): Embedding(43, 400)
  (lstm): LSTM(400, 256, num_layers=2, batch_first=True, dropout=0.05, bidirectional=True)
  (dropout): Dropout(p=0.3, inplace=False)
  (fc): Linear(in_features=512, out_features=1, bias=True)
  (sig): Sigmoid()
)

In [97]:
EPOCHS = 10
optimizer = AdamW(LSTM_model.parameters(), lr=0.001, correct_bias=False)
total_steps = len(train_data_loader) * EPOCHS
scheduler = get_linear_schedule_with_warmup(
  optimizer,
  num_warmup_steps=0,
  num_training_steps=total_steps
)
loss_fn = nn.BCELoss().to(device)

In [98]:
%%time
LSTM_history = train(LSTM_model, EPOCHS, train_data_loader, loss_fn, optimizer, scheduler)

Epoch 1/10 Metrics
{   'train': {'accuracy': 0.5166561910747958, 'loss': 0.7208782096004965},
    'validation': {'accuracy': 0.5577889447236181, 'loss': 0.6840020155906678}}
----------
Epoch 2/10 Metrics
{   'train': {'accuracy': 0.5034569453174105, 'loss': 0.6989296504001522},
    'validation': {'accuracy': 0.49748743718592964, 'loss': 0.7000772547721863}}
----------
Epoch 3/10 Metrics
{   'train': {'accuracy': 0.5009428032683847, 'loss': 0.6987169163310947},
    'validation': {'accuracy': 0.46733668341708545, 'loss': 0.698102753162384}}
----------
Epoch 4/10 Metrics
{   'train': {'accuracy': 0.5172847265870522, 'loss': 0.6949883004528793},
    'validation': {'accuracy': 0.5577889447236181, 'loss': 0.6915875363349915}}
----------
Epoch 5/10 Metrics
{   'train': {'accuracy': 0.4952859836580767, 'loss': 0.6985239266750202},
    'validation': {'accuracy': 0.4472361809045226, 'loss': 0.7019394755363464}}
----------
Epoch 6/10 Metrics
{   'train': {'accuracy': 0.51414204902577, 'loss': 0.6

In [99]:
plot(LSTM_history)

In [ ]:
eval_model(LSTM_model, loss_fn, sets=['test'])

{'test': {'accuracy': 0.4472361809045226, 'loss': 1.054909760951996}}